> Created to address this issue: https://github.com/turtlemonvh/ionic-spark-utils/issues/15

To set up, first create a cluster and attach each of these libraries.

```
# Maven coordinates
com.ionic:ionic-sdk:2.6.0
org.slf4j:slf4j-api:1.7.30
org.slf4j:slf4j-simple:1.7.30

# Upload Jar from releases
https://github.com/turtlemonvh/ionic-spark-utils/releases/tag/v0.0.1
```

The cluster "libraries" page should look like this:

![databricks-libraries](databricks-libraries.png)

*Note:*

To upload this image, I used: `databricks fs cp ~/Desktop/Screen\ Shot\ 2020-05-22\ at\ 12.35.19\ AM.png dbfs:/FileStore/ionic-spark-utils-demo/libraries.png`

Locally we need to create a secret.  Use the Databricks CLI.

```bash
# Install
# https://docs.databricks.com/dev-tools/cli/index.html#install-the-cli
$ pip install databricks-cli

# Configure auth
$ databricks configure
Databricks Host (should begin with https://): https://dbc-9da8a959-d077.cloud.databricks.com/
Username: timothy@ionicsecurity.com
Password:
Repeat for confirmation:

# Create secret scope
# https://docs.databricks.com/security/secrets/secret-scopes.html#create-a-databricks-backed-secret-scope
$ databricks secrets create-scope --scope ionic-demo --initial-manage-principal users

# Confirm you can list the contents of the scope
$ databricks secrets list --scope ionic-demo
Key name    Last updated
----------  --------------

# Use the machina cli to create a copy a profile as plaintext, so we can load that into secrets
# https://dev.ionic.com/tools/machina
machina profile move -d O_6t.e.06a7dcec-7f4b-4361-543e-048e4b1a733c -t plaintext -f profile.tmp

# Load that profile into databricks and delete the local unencrypted copy
$ databricks secrets put --scope ionic-demo --key demo-profile --string-value $(cat profile.tmp)
$ rm profile.tmp

# Ensure the new value shows up in the list
$ databricks secrets list --scope ionic-demo
Key name        Last updated
------------  --------------
demo-profile   1590122961171
```

In [3]:
# Now we can make sure we can load that. We'll use python for this step since it's easy.
# Databricks helpfully redacts the secret value so we don't do anything silly.

dbutils.secrets.get(scope="ionic-demo", key="demo-profile")

Out[1]: '[REDACTED]'

In [4]:
# We do want to make sure we can work with this value, though, so let's make sure we can parse the JSON form of the profile 

import json

json.loads(dbutils.secrets.get(scope="ionic-demo", key="demo-profile"))["profiles"][0]["deviceId"]

Out[2]: 'O_6t.e.06a7dcec-7f4b-4361-543e-048e4b1a733c'

In [5]:
%scala
// Now we can start using this to do something interesting
// Let's start by making an agent

import com.ionic.sdk.agent.Agent
import com.ionic.sdk.device.profile.persistor.DeviceProfiles

// Need to mark these as transient because they are not serializable and databricks tries serialize them into the scope available to each executor
// https://www.scala-lang.org/files/archive/spec/2.11/11-annotations.html#java-platform-annotations
// https://docs.oracle.com/javase/specs/jls/se8/html/jls-8.html#jls-8.3.1.3
// https://docs.oracle.com/javase/8/docs/api/java/beans/Transient.html
@transient val profileJson = dbutils.secrets.get(scope="ionic-demo", key="demo-profile")
@transient val deviceProfiles = new DeviceProfiles(profileJson)
@transient val a = new Agent(deviceProfiles)

import com.ionic.sdk.agent.Agent
import com.ionic.sdk.device.profile.persistor.DeviceProfiles
profileJson: String = [REDACTED]
deviceProfiles: com.ionic.sdk.device.profile.persistor.DeviceProfiles = com.ionic.sdk.device.profile.persistor.DeviceProfiles@581cf35c
a: com.ionic.sdk.agent.Agent = com.ionic.sdk.agent.Agent@57bdbdaf

In [6]:
%scala
// Check our active profile

a.setActiveProfile("O_6t.e.06a7dcec-7f4b-4361-543e-048e4b1a733c")
a.getActiveProfile().getDeviceId()

res0: String = O_6t.e.06a7dcec-7f4b-4361-543e-048e4b1a733c

In [7]:
%scala
// Let's create and fetch a key just for fun

// https://dev.ionic.com/sdk_docs/ionic_platform_sdk/java/version_2.7.0/sdk/com/ionic/sdk/agent/Agent.html
@transient val keyid = a.createKey().getFirstKey().getId() 

keyid: String = O_6tOZPQCOc

In [8]:
%scala
// Fetch

a.getKey(keyid)

res1: com.ionic.sdk.agent.request.getkey.GetKeysResponse = com.ionic.sdk.agent.request.getkey.GetKeysResponse@7048e610

In [9]:
%scala
// Now that we have the basics, we'll move into some Spark specific code

In [10]:
%scala
// Sample data from: https://docs.databricks.com/getting-started/spark/datasets.html#load-sample-data

val ds = spark.read.json("/databricks-datasets/iot/iot_devices.json")

ds: org.apache.spark.sql.DataFrame = [battery_level: bigint, c02_level: bigint ... 13 more fields]

In [11]:
%scala
display(ds)

battery_level,c02_level,cca2,cca3,cn,device_id,device_name,humidity,ip,latitude,lcd,longitude,scale,temp,timestamp
8,868,US,USA,United States,1,meter-gauge-1xbYRYcj,51,68.161.225.1,38.0,green,-97.0,Celsius,34,1458444054093
7,1473,NO,NOR,Norway,2,sensor-pad-2n2Pea,70,213.161.254.1,62.47,red,6.15,Celsius,11,1458444054119
2,1556,IT,ITA,Italy,3,device-mac-36TWSKiT,44,88.36.5.1,42.83,red,12.83,Celsius,19,1458444054120
6,1080,US,USA,United States,4,sensor-pad-4mzWkz,32,66.39.173.154,44.06,yellow,-121.32,Celsius,28,1458444054121
4,931,PH,PHL,Philippines,5,therm-stick-5gimpUrBB,62,203.82.41.9,14.58,green,120.97,Celsius,25,1458444054122
3,1210,US,USA,United States,6,sensor-pad-6al7RTAobR,51,204.116.105.67,35.93,yellow,-85.46,Celsius,27,1458444054122
3,1129,CN,CHN,China,7,meter-gauge-7GeDoanM,26,220.173.179.1,22.82,yellow,108.32,Celsius,18,1458444054123
0,1536,JP,JPN,Japan,8,sensor-pad-8xUD6pzsQI,35,210.173.177.1,35.69,red,139.69,Celsius,27,1458444054123
3,807,JP,JPN,Japan,9,device-mac-9GcjZ2pw,85,118.23.68.227,35.69,green,139.69,Celsius,13,1458444054124
7,1470,US,USA,United States,10,sensor-pad-10BsywSYUF,56,208.109.163.218,33.61,red,-111.89,Celsius,26,1458444054125


In [12]:
%scala
// Let's assume that the device name is encrypted

import com.ionic.sparkutil.KeyServicesCache;
import com.ionic.sdk.key.KeyServices;

// I'm leaving this string outside because I was having issues getting dbutils to behave inside agentFactory
val profileJson = dbutils.secrets.get(scope="ionic-demo", key="demo-profile")

// On each spark worker we'll load the secret from databricks and create an agent
// This has some overhead, but not too much for larger transform operations
// We're considering other strategies to reduce overhead going forward

def agentFactory(): KeyServices = {
  val deviceProfiles = new DeviceProfiles(profileJson)
  val threadLocalAgent = new Agent(deviceProfiles)
  threadLocalAgent.setActiveProfile("O_6t.e.06a7dcec-7f4b-4361-543e-048e4b1a733c")
  new KeyServicesCache(threadLocalAgent)
}

import com.ionic.sparkutil.KeyServicesCache
import com.ionic.sdk.key.KeyServices
profileJson: String = [REDACTED]
agentFactory: ()com.ionic.sdk.key.KeyServices

In [13]:
%scala
import com.ionic.sparkutil.Transformers;

val encryptedDF = ds.transform(Transformers.Encrypt(
  encryptCols = List("device_name"),
  decryptCols = List(),
  agentFactory = agentFactory
))
.drop("device_name")
.withColumnRenamed("ionic_enc_device_name", "device_name")

encryptedDF.persist

display(encryptedDF)

battery_level,c02_level,cca2,cca3,cn,device_id,humidity,ip,latitude,lcd,longitude,scale,temp,timestamp,device_name
8,868,US,USA,United States,1,51,68.161.225.1,38.0,green,-97.0,Celsius,34,1458444054093,~!3!O_6tfcCAiFM!SzwmAJ5V0sJaGeLHsn2yavNhYG8fRzsvb+3YyLMN7/6sfSLor9erKzHAoBvda8hCbjgKMA!
7,1473,NO,NOR,Norway,2,70,213.161.254.1,62.47,red,6.15,Celsius,11,1458444054119,~!3!O_6tfcCAiFM!KpXJvZpCScKqUfWE1eZ2ijt7y1L40y2bLIrAwhn+HaNSf0ge8+7tRWHuu59QxcuQ6g!
2,1556,IT,ITA,Italy,3,44,88.36.5.1,42.83,red,12.83,Celsius,19,1458444054120,~!3!O_6tfcCAiFM!AUit67vyTpSt6B6oW1YYhf1QqQQlksrVPMbWMh4XkaOt/SqSU1103uoKIWHBd558uj0K!
6,1080,US,USA,United States,4,32,66.39.173.154,44.06,yellow,-121.32,Celsius,28,1458444054121,~!3!O_6tfcCAiFM!ZD3wsU5jDsDLvr0u7nkEz/qHg8uZYSvSaIZ5C7dBZbJJsZXrLYD3p9FXzKclgDGf1g!
4,931,PH,PHL,Philippines,5,62,203.82.41.9,14.58,green,120.97,Celsius,25,1458444054122,~!3!O_6tfcCAiFM!Gauj0dTrJCQ3I1q2sz6jaGrhgTC2RUxnEH3XD1o/NKb+X+nRIL3SojWEQOfTjeXHwJeqf5A!
3,1210,US,USA,United States,6,51,204.116.105.67,35.93,yellow,-85.46,Celsius,27,1458444054122,~!3!O_6tfcCAiFM!iVXA4Mud6WtoECKiANMQQsmBYLTqzzJVeej3UCOlBcvObEjRPDWkTHN+mJuEPsjZ/4y5tLs!
3,1129,CN,CHN,China,7,26,220.173.179.1,22.82,yellow,108.32,Celsius,18,1458444054123,~!3!O_6tfcCAiFM!a0aoLA9r5/8w+DKDR3CVnowX5smqE3B1yqxDDfAaBFRcvW8rCHs20GYwdvB+cy1DrSKElQ!
0,1536,JP,JPN,Japan,8,35,210.173.177.1,35.69,red,139.69,Celsius,27,1458444054123,~!3!O_6tfcCAiFM!ja9l0k7ktqplXJas6oDmFkyTf1Ck72EHUXR64jMF+4x2kTe46iOjJpYk15YUwLlNlO44aog!
3,807,JP,JPN,Japan,9,85,118.23.68.227,35.69,green,139.69,Celsius,13,1458444054124,~!3!O_6tfcCAiFM!fyKnCo5u2H4Lcfpq5cZcuYUiMe2W05MZ7rJR9e9zx+QLhaitQHOcAALrQrc4bSu11DAh!
7,1470,US,USA,United States,10,56,208.109.163.218,33.61,red,-111.89,Celsius,26,1458444054125,~!3!O_6tfcCAiFM!FuAD4l5H6l03g6gAvsDAQ7Tg2pZxXuo83MLtsSzvoU6kRQBXFC97KLCX8oBkzXGmOAeBTks!


In [14]:
%scala
// Now let's decrypt

val decryptedDF = encryptedDF.transform(Transformers.Encrypt(
  encryptCols = List(),
  decryptCols = List("device_name"),
  agentFactory = agentFactory
))
.drop("device_name")
.withColumnRenamed("ionic_dec_device_name", "device_name")

decryptedDF.persist

display(decryptedDF)

battery_level,c02_level,cca2,cca3,cn,device_id,humidity,ip,latitude,lcd,longitude,scale,temp,timestamp,device_name
8,868,US,USA,United States,1,51,68.161.225.1,38.0,green,-97.0,Celsius,34,1458444054093,meter-gauge-1xbYRYcj
7,1473,NO,NOR,Norway,2,70,213.161.254.1,62.47,red,6.15,Celsius,11,1458444054119,sensor-pad-2n2Pea
2,1556,IT,ITA,Italy,3,44,88.36.5.1,42.83,red,12.83,Celsius,19,1458444054120,device-mac-36TWSKiT
6,1080,US,USA,United States,4,32,66.39.173.154,44.06,yellow,-121.32,Celsius,28,1458444054121,sensor-pad-4mzWkz
4,931,PH,PHL,Philippines,5,62,203.82.41.9,14.58,green,120.97,Celsius,25,1458444054122,therm-stick-5gimpUrBB
3,1210,US,USA,United States,6,51,204.116.105.67,35.93,yellow,-85.46,Celsius,27,1458444054122,sensor-pad-6al7RTAobR
3,1129,CN,CHN,China,7,26,220.173.179.1,22.82,yellow,108.32,Celsius,18,1458444054123,meter-gauge-7GeDoanM
0,1536,JP,JPN,Japan,8,35,210.173.177.1,35.69,red,139.69,Celsius,27,1458444054123,sensor-pad-8xUD6pzsQI
3,807,JP,JPN,Japan,9,85,118.23.68.227,35.69,green,139.69,Celsius,13,1458444054124,device-mac-9GcjZ2pw
7,1470,US,USA,United States,10,56,208.109.163.218,33.61,red,-111.89,Celsius,26,1458444054125,sensor-pad-10BsywSYUF
